#### 사용자 정의 모델 클래스
- 부모클래스 : nn.Module
- 필수오버라이딩 : 
    * __init__() : 모델 층 구성 즉, 설계
    * forward() : 순방 향학습 진행 코드 구현 

In [1]:
# 모듈 로딩

import torch                                    # 텐서 관련 모듈
import torch.nn as nn                           # 인공신경망 관련 모듈
import torch.nn.functional as F                 # 손실함수 활성화함수 등 인공신경망 관련 함수 모듈
import torch.optim as optim                     # 최적화 관련 모듈(가중치, 절편 빠르게 찾아주는 알고리즘 (경사하강법))
from torchinfo import summary                   # 모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *           # 회귀 성능 지표 관련 모듈 
from torchmetrics.classification import *       # 분류 성능 지표 관련 모듈

import pandas as pd                 # 데이터 파일 분석 관련 모듈



In [2]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- [기본] 신경망클래스 <hr>
    * 입력층 - 입력 : 피쳐수 고정
    * 출력층 - 출력 : 타겟수 고정
    * 은닉층 - 일단 고정(요건 내맘대로 설정)

In [3]:
## 모델 설계
# 사용 데이터셋 : 피쳐 4개, 타겟 1개, [회귀/분류]
# 입력층 : 입력 4개    출력 20개    AF ReLU (시그모이드 안쓰는 이유 - 입력한 정보가 소실되어 >>> 좀 보완하기 위해ReLU를 쓰는데 이것도 소실이 있긴하다) > 그 죽은거 회생하는거 ReLU 였지??
# 은닉층 : 입력 20개   출력 100개   AF ReLU
# 출력층 : 입력 100개  출력 1개     AF X(회귀일땐 필요없음), [Sigmoid or softmax(다중분류) 분류일때 사용 >> 확률로 출력]

In [4]:
# 클래스 생성
class MyModel(nn.Module): # 부모 클래스 상속 
    
    # 인스턴스/객체 생성 시 자동 호출되는 메서드
    def __init__(self):
        # 부모클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(4,20)       # W 4 + b 1 => 1P , 5 * 20 = 100개 변수, 지금 현재 4개로 설정
        self.hidden_layer = nn.Linear(20,100)    # W 20 + b 1 => 1P , 21 * 100 = 2100개 변수
        self.output_layer = nn.Linear(100,1)     # W 100 + b 1 => 1P , 101 * 1 = 101개 변수

    # ---------------------------------------------------------------- 여기까지는 모델 구성 설계까지만

    # 순반향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋 
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x) # 1개 퍼셉트론 : y= x1W1 + x2W2 + x3W3 + x4W4 + b * 20개 [피쳐,가중치(W)] 4개 절편(b)1개
        y= F.relu(y)  # relu의 결과는  0 <= y --> relu도 완벽하지않음 데이터 소실 적다는거지 없다는건 아님 >> 죽은 relu 발생  ==> 보완 >> leakyReLu

        y=self.hidden_layer(y) # 1개 퍼셉트론 : y= x1W1 + x2W2 + ... + x20W20 + b * 100개  [피쳐,가중치(W)] 20개 절편(b)1개
        y= F.relu(y)
        
        return self.output_layer(y) # 1개 퍼셉트론 : y= x1W1 + x2W2 + ... + x100W100 + b * 1개  [피쳐,가중치(W)] 100개 절편(b)1개 >> 출력
        

In [5]:
# 입력 피쳐 수 동적인 모델
class MyModel2(nn.Module): # 부모 클래스 상속 
    
    # 인스턴스/객체 생성 시 자동 호출되는 메서드
    def __init__(self, in_feature):
        # 부모클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_feature,20)       # W [in_featur개수] + b 1 => 1P , (n+1) * 20 = 100개 변수, in_featur개수 
        self.hidden_layer = nn.Linear(20,100)    # W 20 + b 1 => 1P , 21 * 100 = 2100개 변수
        self.output_layer = nn.Linear(100,1)     # W 100 + b 1 => 1P , 101 * 1 = 101개 변수

    # ---------------------------------------------------------------- 여기까지는 모델 구성만

    # 순반향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋 
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x) # 1개 퍼셉트론 : y= x1W1 + x2W2 + x3W3 + x4W4 + b * 20개 [피쳐,가중치(W)] 4개 절편(b)1개
        y= F.relu(y)  # relu의 결과는  0 <= y 

        y=self.hidden_layer(y) # 1개 퍼셉트론 : y= x1W1 + x2W2 + ... + x20W20 + b * 100개  [피쳐,가중치(W)] 20개 절편(b)1개
        y= F.relu(y)
        
        return self.output_layer(y) # 1개 퍼셉트론 : y= x1W1 + x2W2 + ... + x100W100 + b * 1개  [피쳐,가중치(W)] 100개 절편(b)1개 >> 출력
        

In [6]:
# 입력 피쳐 수, 은닉층 퍼셉트론 수 동적인 모델
class MyModel3(nn.Module): # 부모 클래스 상속 
    
    # 인스턴스/객체 생성 시 자동 호출되는 메서드
    def __init__(self, in_feature, in_out, h_out):
        # 부모클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_feature,in_out)     
        self.hidden_layer = nn.Linear(in_out,h_out)  
        self.output_layer = nn.Linear(h_out,1)   

    # ---------------------------------------------------------------- 여기까지는 모델 구성만

    # 순반향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋 
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x) 
        y= F.relu(y)  

        y=self.hidden_layer(y) 
        y= F.relu(y)
        
        return self.output_layer(y) 

In [ ]:
# 은닉층의 개수, 입력 피쳐 수, 은닉층 퍼셉트론 수 동적인 모델 - 나의 도전 ㅋㅋ >>> 정확한건 다음 페이지에
class MyModel4(nn.Module): # 부모 클래스 상속 
    
    # 추가적으로 은닉층 갯수와 각 은닉층의 퍼셉트론의 개수가 있어야한다.
    def __init__(self, in_feature, in_out, h_out, *args):
        # 부모클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        
        # 입력층
        self.input_layer = nn.Linear(in_feature,in_out)  
        
        # 은닉층
        if args<=0:
            pass
        else:
            for i in range(1, args+1):
                print(f'은닉{i}층')
                self.hidden_layer = nn.Linear(in_out,h_out)  
        
        # 출력층
        self.output_layer = nn.Linear(h_out,1)   

    # ---------------------------------------------------------------- 여기까지는 모델 구성 설계만

    # 순반향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋 
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x) 
        y= F.relu(y)  

        y=self.hidden_layer(y) 
        y= F.relu(y)
        
        return self.output_layer(y) 

In [7]:
## 모델 인스턴스 생성
m1 = MyModel()

In [8]:
# 모델 파라미터 즉, W와 b 확인
for m in m1.named_parameters(): print(m)

   # requires_grad=True 변경이 되는 데이터 > 변수 ,  변경이 되면 안되는 데이터 > 상수

('input_layer.weight', Parameter containing:
tensor([[ 0.2576, -0.2207, -0.0969,  0.2347],
        [-0.4707,  0.2999, -0.1029,  0.2544],
        [ 0.0695, -0.0612,  0.1387,  0.0247],
        [ 0.1826, -0.1949, -0.0365, -0.0450],
        [ 0.0725, -0.0020,  0.4371,  0.1556],
        [-0.1862, -0.3020, -0.0838, -0.2157],
        [-0.1602,  0.0239,  0.2981,  0.2718],
        [-0.4888,  0.3100,  0.1397,  0.4743],
        [ 0.3300, -0.4556, -0.4754, -0.2412],
        [ 0.4391, -0.0833,  0.2140, -0.2324],
        [ 0.4906, -0.2115,  0.3750,  0.0059],
        [-0.2634,  0.2570, -0.2654,  0.1471],
        [-0.1444, -0.0548, -0.4807, -0.2384],
        [ 0.2713, -0.1215,  0.4980,  0.4008],
        [-0.0234, -0.3337,  0.3045,  0.1552],
        [-0.3232,  0.3248,  0.3036,  0.4434],
        [-0.2803, -0.0823, -0.0097,  0.0730],
        [-0.3795, -0.3548,  0.2720, -0.1172],
        [ 0.2442,  0.0285,  0.1642,  0.1099],
        [ 0.1818,  0.2479, -0.4631,  0.2517]], requires_grad=True))
('input_layer

In [9]:
# 학습 진행 ==> 모델 인스턴스(데이터)
# 임의의 데이터
dataTS = torch.FloatTensor([[1,3,5,7],[2,4,6,8]]) # 넣을려는 데이터의 피쳐의 갯수와 모델의 피쳐의 개수가 일치해야 한다.
targetTS= torch.FloatTensor([[4],[5]])

# 학습
pre_y = m1(dataTS)

pre_y

calling forward()


tensor([[-0.1395],
        [-0.1858]], grad_fn=<AddmmBackward0>)

In [11]:
m3 = MyModel3(10,100,70)
m3 

MyModel3(
  (input_layer): Linear(in_features=10, out_features=100, bias=True)
  (hidden_layer): Linear(in_features=100, out_features=70, bias=True)
  (output_layer): Linear(in_features=70, out_features=1, bias=True)
)